In [1]:
#reference to: /home/lq53/mir_repos/BBOP/random_tests/25May_tuning_retry/scan_250508_for_good_mini_path.ipynb

In [2]:
import sys
import os
sys.path.append(os.path.abspath('../..'))


# from scan_engine.status_fields_config_24summ import STATUS_FIELDS_CONFIG
from status_fields_config_oct3v1_brws_250525 import STATUS_FIELDS_CONFIG

from utlis.scan_engine_utlis.scan_eng_big_utlis import log_folder_to_parquet_sep

if __name__ == "__main__":
    base_folder = "/data/big_rim/rsync_dcc_sum/Oct3V1" #"/hpc/group/tdunn/Bryan_Rigs/BigOpenField/24summ"  # Replace with your base folder
    # save_path = os.path.join(base_folder, 'paret')
    failed_paths_file = '/data/big_rim/rsync_dcc_sum/Oct3V1/sync_failed_brws.txt'  # File containing failed paths

    force_rescan_rec_files = [
        # ('2023-10-01', '001'),
        # ('2023-10-02', '002'),
        # Add more as needed
    ]
    rescan_threshold_days = 0.001 # 7 days, but guess if i mess up i can just change it to automatically rescan all, smile... #0.1

    log_folder_to_parquet_sep(base_folder, failed_paths_file, STATUS_FIELDS_CONFIG,
                              force_rescan_rec_files=force_rescan_rec_files,
                              rescan_threshold_days=rescan_threshold_days)


Log for 24Anshu_f_paint_2mice_2 saved at /data/big_rim/rsync_dcc_sum/Oct3V1/2024_10_03_micecolor_test/24Anshu_f_paint_2mice_2/folder_log.parquet
Log for 24Anshu_f_paint_2mice saved at /data/big_rim/rsync_dcc_sum/Oct3V1/2024_10_03_micecolor_test/24Anshu_f_paint_2mice/folder_log.parquet
Log for 24Anshu_f_bleach_2mice saved at /data/big_rim/rsync_dcc_sum/Oct3V1/2024_10_03_micecolor_test/24Anshu_f_bleach_2mice/folder_log.parquet
Log for 2social_mini_0605pmc_single_15_38 saved at /data/big_rim/rsync_dcc_sum/Oct3V1/2024_10_30/2social_mini_0605pmc_single_15_38/folder_log.parquet
Log for 2social_mini_0605pmc_single_15_00 saved at /data/big_rim/rsync_dcc_sum/Oct3V1/2024_10_30/2social_mini_0605pmc_single_15_00/folder_log.parquet
Log for 2social_mini_0605pmc_single_15_18 saved at /data/big_rim/rsync_dcc_sum/Oct3V1/2024_10_30/2social_mini_0605pmc_single_15_18/folder_log.parquet
Log for 20241217v1l23re1_p20241217v1l23BE0 saved at /data/big_rim/rsync_dcc_sum/Oct3V1/2025_05_21/20241217v1l23re1_p20241

In [3]:
import sys, os
sys.path.append(os.path.abspath('../..'))
from utlis.scan_engine_utlis.scan_engine_utlis import read_all_parquet_files
base_folder = "/data/big_rim/rsync_dcc_sum/Oct3V1" 
all_df = read_all_parquet_files(base_folder)

In [4]:
import pyarrow.compute as pc
from functools import reduce
table = all_df #combined_df

# quality_mask = pc.or_(
#     pc.equal(table['quality'], 'good'),
#     pc.equal(table['quality'], 'good_vein')
# )

conditions = [
   pc.equal(table['mini_rec_sync'], '1'),
   pc.equal(table['social'], '0'),
   pc.equal(table['date_folder'], '2025_05_16'),
#    pc.equal(table['quality'], 'good_vein')
# quality_mask
]

filter_mask = reduce(pc.and_, conditions)
# Apply the filter and print the results
post_ana = table.filter(filter_mask)

# Print each row of the filtered table
print(post_ana.to_pandas())  # This will display the filtered data in a familiar pandas-like format


  mir_generate_param sync mini_6cam_map dropf_handle com com_vis social  \
0                  1    1             1            0   1       1      0   
1                  1    1             1            0   1       1      0   
2                  1    1             1            0   1       1      0   

  miniscope test after_oxytocin before_oxytocin dannce dannce_vis  \
0         1    0              0               0      1          1   
1         1    0              0               0      1          1   
2         1    0              0               0      1          1   

  mini_rec_sync mini_rec_sync_com                      rec_file  \
0             1                 0        20240303PMCBE0r1coated   
1             1                 0  20240303PMCBE0r1coated_14_53   
2             1                 0          20241216V1RE1F_15_16   

                    scan_time  \
0  2025-07-03T11:46:30.166670   
1  2025-07-03T11:46:30.182600   
2  2025-07-03T11:46:30.190097   

                    

In [7]:
# Convert and flatten the ChunkedArray
rec_paths = post_ana["rec_path"].to_pylist()

# Print each path
for path in rec_paths:
    # Each element is likely a list with one string, e.g., ['/some/path']
    print(path[0] if isinstance(path, list) else path)


/data/big_rim/rsync_dcc_sum/Oct3V1/2025_05_16/20240303PMCBE0r1coated
/data/big_rim/rsync_dcc_sum/Oct3V1/2025_05_16/20240303PMCBE0r1coated_14_53
/data/big_rim/rsync_dcc_sum/Oct3V1/2025_05_16/20241216V1RE1F_15_16


In [5]:
import site

# this both inserts the directory *and* processes any .pth files in it
site.addsitedir(
    "/home/lq53/miniconda3/envs/neuroposelib/lib/python3.9/site-packages"
)

# now Python has read __editable__.neuroposelib-0.2.dev0.pth
# and added your repo path automatically.
import neuroposelib
print("✅ loaded from:", neuroposelib.__file__)

✅ loaded from: /home/lq53/mir_repos/dappy_24_nov/neuroposelib/src/neuroposelib/__init__.py


In [ ]:
#!/usr/bin/env python3
import os
import sys
import glob
import json
import pandas as pd

# --- Adjust these to match your loaded Jupyter variables or define here if needed ---
#   post_ana: a PyArrow Table or Pandas DataFrame containing at least 'date_folder' and 'rec_file'
#   base_folder: str, e.g. "/data/big_rim/rsync_dcc_sum/Oct3V1"
#   pickle_path: str, path to your datastruct.p file
#   key:         str, unique suffix for your output files (e.g. "v2_angfix")
#   dry_run:     bool, set True to only log actions
#
# Example:
# base_folder = "/data/big_rim/rsync_dcc_sum/Oct3V1"
# pickle_path = "/home/lq53/.../datastruct.p"
# key         = "v2_angfix"
# dry_run     = False
#
# batch_export_and_csv(post_ana, base_folder, pickle_path, key, dry_run)

# path to your datastruct.p (you can override this in your notebook before calling)
pickle_path = "/home/lq53/mir_repos/dappy_24_nov/2505_rerun/250506_ang_pos_shank/datastruct.p"
# "/home/lq53/mir_repos/dappy_24_nov/byws_version/250429_ang_pos/datastruct.p"
# "/home/lq53/mir_repos/dappy_24_nov/byws_version/250116_wav_ffix_ang_pos/50_p/datastruct.p"

# 1) Insert the other env's site-packages so you can import neuroposelib
OTHER_ENV = "/home/lq53/miniconda3/envs/neuroposelib"
lib_dir   = os.path.join(OTHER_ENV, "lib")
cands     = glob.glob(os.path.join(lib_dir, "python*/site-packages"))
if not cands:
    raise RuntimeError(f"No site-packages found in {lib_dir}")
site_pkgs = cands[0]
print(f"→ adding {site_pkgs} to sys.path")
sys.path.insert(0, site_pkgs)

# 2) Import your utility functions
from utlis.sync_utlis.dappy_align_utlis import (
    load_new_filtered_data_from_h5,
    export_aligned_data_from_paths
)
from utlis.sync_utlis.mini_dannce_sync import generate_frame_mapping


def batch_export_and_csv(post_ana, base_folder, pickle_path, key, dry_run=False):
    """
    post_ana: Arrow Table or DataFrame containing at least 'date_folder' and 'rec_file'
    base_folder: root path for rec sessions
    pickle_path: datastruct.p for export_aligned_data
    key:         unique suffix to stick on your output files
    dry_run:     if True, only logs actions
    """
    # convert Arrow table to pandas DF if needed
    if not isinstance(post_ana, pd.DataFrame):
        try:
            df = post_ana.to_pandas()
        except AttributeError:
            raise ValueError("post_ana must be a pyarrow.Table or pandas.DataFrame")
    else:
        df = post_ana.copy()

    # build absolute rec paths
    df['rec_path'] = df.apply(
        lambda r: os.path.join(base_folder, r['date_folder'], r['rec_file']),
        axis=1
    )
    

    for _, row in df.iterrows():
        rec_path  = row['rec_path']
        mir_dir   = os.path.join(rec_path, "MIR_Aligned")
        frame_map = os.path.join(mir_dir, "frame_mapping.json")
        base_name = f"filtered_data_{key}"
        out_h5    = os.path.join(mir_dir, f"{base_name}.h5")
        out_csv   = os.path.join(mir_dir, f"{base_name}.csv")

        print(f"\n--- Processing {rec_path} ---")

        # skip already processed
        if os.path.exists(out_csv):
            print(f"→ skip, CSV exists at {out_csv}")
            continue

        if dry_run:
            print(f"[DRY RUN] mkdir        {mir_dir}")
            print(f"[DRY RUN] ensure map    {frame_map}")
            print(f"[DRY RUN] export HDF5   {out_h5}")
            print(f"[DRY RUN] write CSV     {out_csv}")
            continue

        # create output dir
        os.makedirs(mir_dir, exist_ok=True)

        # ensure mapping JSON exists and non-empty
        regen = True
        if os.path.exists(frame_map):
            with open(frame_map, 'r') as f:
                mapping = json.load(f)
            if mapping:
                regen = False
            else:
                print("→ existing mapping empty, regenerating...")
        else:
            print("→ no mapping.json, generating...")

        if regen:
            mini_path = os.path.join(rec_path, "My_V4_Miniscope")
            try:
                generate_frame_mapping(
                    rec_path,
                    mini_path,
                    start_frame=0,
                    end_frame=500,
                    threshold_mini=15,
                    threshold_sixcam=3
                )
            except Exception as e:
                print(f"⚠️ generation error: {e}")
                continue
            if not os.path.exists(frame_map):
                print(f"⚠️ mapping.json still missing after generate for {rec_path}")
                continue

        # export HDF5
        try:
            export_aligned_data_from_paths(
                pickle_path,
                rec_path,
                frame_map,
                out_h5
            )
        except Exception as e:
            print(f"⚠️ export error: {e}")
            continue
        if not os.path.exists(out_h5):
            print(f"⚠️ no HDF5 at {out_h5}, skipping CSV")
            continue

        # load & write CSV
        try:
            df_filt = load_new_filtered_data_from_h5(out_h5)
            df_filt.to_csv(out_csv, index=False)
            print(f" → CSV saved to: {out_csv}")
        except Exception as e:
            print(f"⚠️ CSV write error: {e}")


# If you want to run from a notebook or REPL, just call:
batch_export_and_csv(post_ana, base_folder, pickle_path, key="250506_ang_pos", dry_run=False)


→ adding /home/lq53/miniconda3/envs/neuroposelib/lib/python3.9/site-packages to sys.path

--- Processing /data/big_rim/rsync_dcc_sum/Oct3V1/2025_05_16/20240303PMCBE0r1coated ---
No data found for rec_path: /data/big_rim/rsync_dcc_sum/Oct3V1/2025_05_16/20240303PMCBE0r1coated/DANNCE/predict00/save_data_AVG.mat
⚠️ no HDF5 at /data/big_rim/rsync_dcc_sum/Oct3V1/2025_05_16/20240303PMCBE0r1coated/MIR_Aligned/filtered_data_250506_ang_pos.h5, skipping CSV

--- Processing /data/big_rim/rsync_dcc_sum/Oct3V1/2025_05_16/20240303PMCBE0r1coated_14_53 ---
No data found for rec_path: /data/big_rim/rsync_dcc_sum/Oct3V1/2025_05_16/20240303PMCBE0r1coated_14_53/DANNCE/predict00/save_data_AVG.mat
⚠️ no HDF5 at /data/big_rim/rsync_dcc_sum/Oct3V1/2025_05_16/20240303PMCBE0r1coated_14_53/MIR_Aligned/filtered_data_250506_ang_pos.h5, skipping CSV

--- Processing /data/big_rim/rsync_dcc_sum/Oct3V1/2025_05_16/20241216V1RE1F_15_16 ---
No data found for rec_path: /data/big_rim/rsync_dcc_sum/Oct3V1/2025_05_16/20241216